# Statistical Analysis of Excel Data
This notebook performs data import, summary statistics, normality tests, ANOVA/Friedman tests, post-hoc analysis, and effect size calculations for two channels across different conditions.

In [39]:
import pandas as pd
from IPython.display import display
from scipy.stats import f_oneway, shapiro, ttest_rel, wilcoxon
import pingouin as pg
import numpy as np
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import scikit_posthocs as sp
from statsmodels.sandbox.stats.multicomp import multipletests
import scipy.stats as stats  # Import scipy.stats

In [40]:
# Path to your Excel file
excel_path = 'statistike.xlsx'
df_all = pd.read_excel(excel_path, sheet_name="Statistike vseh obravnav pacien")
display(df_all.head())

,način,ID,Kanal 1 Povprečje delovanja,Kanal 1 Odklon od povprečja delovanja,Column1,Kanal 1 Vrh,Kanal 2 Povprečje delovanja,Kanal 2 Odklon od povprečja delovanja,Column2,Kanal 2 Vrh,Kanal 1 Povprečje mirovanja,Kanal 1 Odklon od povprečja mirovanja,Column3,Kanal 1 Minimum,Kanal 2 Povprečje mirovanja,Kanal 2 Odklon od povprečja mirovanja,Column4,Kanal 2 Minimum
0,nosnos,1354,3.536849,0.621890,0.175832,10.9,4.366164,0.874796,0.200358,10.90,3.695455,0.805859,0.218068,1.27,4.669596,0.945205,0.202417,1.53
1,ustpripora,1354,2.243514,0.582542,0.259656,5.4,1.592973,0.544719,0.341951,5.41,1.986578,0.458973,0.231037,0.74,1.460199,0.441143,0.302112,0.56
2,ššš,1354,2.157181,0.503344,0.233334,6.3,1.670940,0.484776,0.290122,7.40,1.954916,0.404280,0.206802,0.84,1.862155,0.565431,0.303643,0.57
3,sss,1354,2.524362,0.496311,0.196608,5.3,2.017450,0.572784,0.283915,5.80,2.384983,0.476827,0.199929,1.11,1.905797,0.516088,0.270799,0.87
4,popek,1354,13.964437,2.621669,0.187739,55.0,19.255364,4.528746,0.235194,96.00,19.278389,6.835292,0.354557,5.80,28.180705,11.823276,0.419552,5.40


In [41]:
pivot_df = df_all.pivot_table(
    index=['ID'],
    columns='način',
    values=[
        'Kanal 1 Povprečje delovanja',
        'Kanal 2 Povprečje delovanja'
    ],
    aggfunc='first'
)
display(pivot_df.head())

Kanal 1 Povprečje delovanja                                             \
način                      nosnos        pas      popek       sss ustpripora   
ID                                                                             
804                      3.005338   2.932945   3.038069  2.991034   2.754247   
1212                     3.543046   2.079028   2.670000  1.054558   2.547770   
1252                     7.087708   5.338095   4.745411  4.209662   5.123649   
1354                     3.536849  10.531921  13.964437  2.524362   2.243514   
2038                     2.019400   2.383219   2.756014  2.234067   2.609306   

                Kanal 2 Povprečje delovanja                                  \
način       ššš                      nosnos        pas      popek       sss   
ID                                                                            
804    3.742877                    3.255676   2.702192   3.470483  2.901379   
1212   2.284595                    3.207351   1.739097   1.971056  1.118503   
1252   9.232886                    6.672708   5.752109   4.903973  4.684324   
1354   2.157181                    4.366164  13.263245  19.255364  2.017450   
2038   2.004730                    2.475467   2.692945   2.723378  2.609133   

                            
način ustpripora       ššš  
ID                          
804     3.140959  3.937534  
1212    1.947838  1.593446  
1252    6.353919  9.648658  
1354    1.592973  1.670940  
2038    2.929861  2.523108

In [42]:
def calculate_rms(group):
    """Calculate RMS for each column in the group."""
    rms_values = {}
    for column in group.columns:
        if column in group.columns:
            squared_values = group[column] ** 2
            mean_squared = squared_values.mean()
            rms = np.sqrt(mean_squared)
            rms_values[column] = rms
        else:
            rms_values[column] = None
    return pd.Series(rms_values)

In [43]:
# Group by 'način' and calculate RMS
rms_by_nacin = df_all.groupby('način')[['Kanal 1 Povprečje delovanja', 'Kanal 2 Povprečje delovanja']].apply(calculate_rms)
display(rms_by_nacin)

,Kanal 1 Povprečje delovanja,Kanal 2 Povprečje delovanja
način,,
nosnos,4.817517,5.942378
pas,5.056939,6.037595
popek,6.002717,7.778452
sss,3.692550,4.174802
ustpripora,3.611618,4.010868
ššš,5.171337,5.594014


In [44]:
# Normality test for each group in 'način' for both channels
for kanal in ['Kanal 1 Povprečje delovanja', 'Kanal 2 Povprečje delovanja']:
    print(f'Normalnost za {kanal}:')
    for name, group in df_all.groupby('način'):
        stat, p = shapiro(group[kanal].dropna())
        print(f"Shapiro-Wilk test za skupino '{name}': stat={stat:.4f}, p-value={p:.4f}")

Normalnost za Kanal 1 Povprečje delovanja:
Shapiro-Wilk test za skupino 'nosnos': stat=0.8430, p-value=0.0041
Shapiro-Wilk test za skupino 'pas': stat=0.8326, p-value=0.0028
Shapiro-Wilk test za skupino 'popek': stat=0.7893, p-value=0.0006
Shapiro-Wilk test za skupino 'sss': stat=0.9126, p-value=0.0713
Shapiro-Wilk test za skupino 'ustpripora': stat=0.9211, p-value=0.1041
Shapiro-Wilk test za skupino 'ššš': stat=0.8306, p-value=0.0026
Normalnost za Kanal 2 Povprečje delovanja:
Shapiro-Wilk test za skupino 'nosnos': stat=0.7505, p-value=0.0002
Shapiro-Wilk test za skupino 'pas': stat=0.7668, p-value=0.0003
Shapiro-Wilk test za skupino 'popek': stat=0.7543, p-value=0.0002
Shapiro-Wilk test za skupino 'sss': stat=0.8796, p-value=0.0174
Shapiro-Wilk test za skupino 'ustpripora': stat=0.9726, p-value=0.8077
Shapiro-Wilk test za skupino 'ššš': stat=0.7911, p-value=0.0006


In [45]:
# Friedman test for Kanal 1
print('\nFriedmanov test za Kanal 1:')
friedman_test = pg.friedman(data=df_all, dv='Kanal 1 Povprečje delovanja', within='način', subject='ID')
display(friedman_test)
# Friedman test for Kanal 2
print('\nFriedmanov test za Kanal 2:')
friedman_test_kanal2 = pg.friedman(data=df_all, dv='Kanal 2 Povprečje delovanja', within='način', subject='ID')
display(friedman_test_kanal2)


Friedmanov test za Kanal 1:


,Source,W,ddof1,Q,p-unc
Friedman,način,0.122,5,12.2,0.032148



Friedmanov test za Kanal 2:


,Source,W,ddof1,Q,p-unc
Friedman,način,0.162571,5,16.257143,0.006147


In [46]:
# Post-hoc Tukey test for Kanal 1
print('\nPost-hoc Tukey test with Bonferroni correction for Kanal 1:')
posthoc_kanal1 = pg.pairwise_tukey(data=df_all, dv='Kanal 1 Povprečje delovanja', between='način')
display(posthoc_kanal1)
# Post-hoc Tukey test for Kanal 2
print('\nPost-hoc Tukey test with Bonferroni correction for Kanal 2:')
posthoc_kanal2 = pg.pairwise_tukey(data=df_all, dv='Kanal 2 Povprečje delovanja', between='način')
display(posthoc_kanal2)


Post-hoc Tukey test with Bonferroni correction for Kanal 1:


,A,B,mean(A),mean(B),diff,se,T,p-tukey,hedges
0,nosnos,pas,4.400738,4.435747,-0.035009,0.710239,-0.049292,1.000000,-0.015156
1,nosnos,popek,4.400738,5.241347,-0.840608,0.710239,-1.183557,0.843848,-0.322474
2,nosnos,sss,4.400738,3.412593,0.988145,0.710239,1.391285,0.732175,0.552850
3,nosnos,ustpripora,4.400738,3.436595,0.964144,0.710239,1.357491,0.752054,0.578177
4,nosnos,ššš,4.400738,4.417245,-0.016507,0.710239,-0.023241,1.000000,-0.006702
5,pas,popek,4.435747,5.241347,-0.805599,0.710239,-1.134265,0.866080,-0.286241
6,pas,sss,4.435747,3.412593,1.023154,0.710239,1.440577,0.702249,0.492240
7,pas,ustpripora,4.435747,3.436595,0.999153,0.710239,1.406783,0.722877,0.505518
8,pas,ššš,4.435747,4.417245,0.018502,0.710239,0.026051,1.000000,0.006899
9,popek,sss,5.241347,3.412593,1.828754,0.710239,2.574842,0.112004,0.760659



Post-hoc Tukey test with Bonferroni correction for Kanal 2:


,A,B,mean(A),mean(B),diff,se,T,p-tukey,hedges
0,nosnos,pas,5.158460,5.087497,0.070963,0.977947,0.072563,1.000000,0.021839
1,nosnos,popek,5.158460,6.322089,-1.163629,0.977947,-1.189869,0.840870,-0.290740
2,nosnos,sss,5.158460,3.761466,1.396994,0.977947,1.428496,0.709677,0.545233
3,nosnos,ustpripora,5.158460,3.797799,1.360661,0.977947,1.391344,0.732140,0.570962
4,nosnos,ššš,5.158460,4.640150,0.518310,0.977947,0.529998,0.994850,0.162962
5,pas,popek,5.087497,6.322089,-1.234592,0.977947,-1.262432,0.804624,-0.299066
6,pas,sss,5.087497,3.761466,1.326031,0.977947,1.355933,0.752958,0.481378
7,pas,ustpripora,5.087497,3.797799,1.289698,0.977947,1.318781,0.774098,0.498162
8,pas,ššš,5.087497,4.640150,0.447347,0.977947,0.457435,0.997431,0.134035
9,popek,sss,6.322089,3.761466,2.560623,0.977947,2.618365,0.101164,0.708882


In [47]:
# Hipoteza 2: Primerjava parov načinov


for kanal in ['Kanal 1 Povprečje delovanja', 'Kanal 2 Povprečje delovanja']:
    print(f"\nAnaliza za {kanal}:")
    for pair in pairs:
        group1 = df_all[df_all['način'] == pair[0]][kanal].dropna()
        group2 = df_all[df_all['način'] == pair[1]][kanal].dropna()
        p1 = shapiro(group1)[1]
        p2 = shapiro(group2)[1]
        print(f"\nPrimerjava {pair[0]} vs {pair[1]}:")
        if p1 > alpha and p2 > alpha:
            t_stat, p_val = ttest_rel(group1.values, group2.values)
            print(f"Paired t-test: t={t_stat:.4f}, p={p_val:.4f} (Bonferroni alpha={bonferroni_alpha:.3f})")
            if p_val < bonferroni_alpha:
                print("Rezultat: statistično značilno")
            else:
                print("Rezultat: ni statistično značilno")
        else:
            stat, p_val = wilcoxon(group1.values, group2.values)
            print(f"Wilcoxon test: stat={stat:.4f}, p={p_val:.4f} (Bonferroni alpha={bonferroni_alpha:.3f})")
            if p_val < bonferroni_alpha:
                print("Rezultat: statistično značilno")
            else:
                print("Rezultat: ni statistično značilno")



Analiza za Kanal 1 Povprečje delovanja:

Primerjava nosnos vs ššš:
Wilcoxon test: stat=89.0000, p=0.5706 (Bonferroni alpha=0.025)
Rezultat: ni statistično značilno

Primerjava ššš vs ustpripora:
Wilcoxon test: stat=84.0000, p=0.4524 (Bonferroni alpha=0.025)
Rezultat: ni statistično značilno

Analiza za Kanal 2 Povprečje delovanja:

Primerjava nosnos vs ššš:
Wilcoxon test: stat=75.0000, p=0.2774 (Bonferroni alpha=0.025)
Rezultat: ni statistično značilno

Primerjava ššš vs ustpripora:
Wilcoxon test: stat=90.0000, p=0.5958 (Bonferroni alpha=0.025)
Rezultat: ni statistično značilno


In [48]:
#HIPOTEZA 3

# Izračunaj povprečne RMS vrednosti za načine 'popek', 'pas' in ostale načine
nacini = ['popek', 'pas']
rms_popek_pas = rms_by_nacin.loc[nacini]
rms_ostali = rms_by_nacin.drop(nacini).mean()
print("RMS za 'popek' in 'pas':")
display(rms_popek_pas)
print("Povprečne RMS vrednosti za ostale načine:")
display(rms_ostali)

RMS za 'popek' in 'pas':


,Kanal 1 Povprečje delovanja,Kanal 2 Povprečje delovanja
način,,
popek,6.002717,7.778452
pas,5.056939,6.037595


Povprečne RMS vrednosti za ostale načine:


Kanal 1 Povprečje delovanja    4.323255
Kanal 2 Povprečje delovanja    4.930516
dtype: float64

In [49]:
# Preveri normalnost za 'pas' in 'popek' za oba kanala
for kanal in ['Kanal 1 Povprečje delovanja', 'Kanal 2 Povprečje delovanja']:
    print(f"\nShapiro-Wilk test za kanal: {kanal}")
    for nacin in ['pas', 'popek']:
        group = df_all[df_all['način'] == nacin][kanal].dropna()
        stat, p_value = shapiro(group)
        print(f"  {nacin}: stat={stat:.4f}, p-value={p_value:.4f}")


Shapiro-Wilk test za kanal: Kanal 1 Povprečje delovanja
  pas: stat=0.8326, p-value=0.0028
  popek: stat=0.7893, p-value=0.0006

Shapiro-Wilk test za kanal: Kanal 2 Povprečje delovanja
  pas: stat=0.7668, p-value=0.0003
  popek: stat=0.7543, p-value=0.0002


Tole ni normalno, zato uporabimo friedman + posthoc dun bonferoni

In [52]:
# Pripravi podatke za Friedmanov test: 'popek', 'pas' in povprečje ostalih načinov za Kanal 1

from pandas import IndexSlice as idx

selected_nacins = ['popek', 'pas']

ostali_nacini = [n for n in rms_by_nacin.index if n not in selected_nacins]

# Ustvari pivot tabelo za izbrane načine in povprečje ostalih

pivot_selected = pivot_df.loc[:, idx['Kanal 1 Povprečje delovanja', selected_nacins]].copy()

pivot_selected.columns = selected_nacins  # Preimenuj stolpce za lažji dostop

pivot_selected['ostali'] = pivot_df.loc[:, idx['Kanal 1 Povprečje delovanja', ostali_nacini]].mean(axis=1)

# Friedmanov test

friedman_stat, friedman_p = stats.friedmanchisquare(
    pivot_selected['popek'],
    pivot_selected['pas'],
    pivot_selected['ostali']

)

print(f"Friedmanov test: stat={friedman_stat:.4f}, p-value={friedman_p:.4f}")

# Post-hoc Dunn-Bonferroni test

# Pripravi podatke v long formatu

long_df = pivot_selected.melt(var_name='nacin', value_name='vrednost')

long_df['ID'] = long_df.index % pivot_selected.shape[0]

dunn_result = sp.posthoc_dunn(
    [long_df[long_df['nacin'] == n]['vrednost'].values for n in ['popek', 'pas', 'ostali']],
    p_adjust='bonferroni'

)

dunn_result.index = ['popek', 'pas', 'ostali']

dunn_result.columns = ['popek', 'pas', 'ostali']

display(dunn_result)

Friedmanov test: stat=3.7000, p-value=0.1572


,popek,pas,ostali
popek,1.000000,0.761921,0.473534
pas,0.761921,1.000000,1.000000
ostali,0.473534,1.000000,1.000000
